<a href="https://colab.research.google.com/github/AhmedThahir/Certificate_Generator_Google_Dev_Club/blob/main/Certificates_Generation_Mail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDSC Certificates Generation + Mail

**Note:**
First test for your own account

Steps
- [ ] Update names string
- [ ] Update emails string
- [ ] Run All Cells
- [ ] Give answers for the required prompts

Made with Python <3 by Ahmed Thahir

In [ ]:
names = """
Ahmed Thahir
"""

In [ ]:
emails = """
f20200198@dubai.bits-pilani.ac.in
"""

# Don't Edit 

In [ ]:
sender_mail = "gdsc@dubai.bits-pilani.ac.in"

In [ ]:
names = list(filter(None, names.split("\n")))
emails = list(filter(None, emails.split("\n")))

if(len(names)!=len(emails)):
    raise ValueError('Unequal number of names and emails')

In [ ]:
event_name = input("Event Name ")
event_date = input("Event Date (YYYY-MM-DD) ")

from datetime import datetime
event_date = datetime.strptime(event_date, '%Y-%m-%d').strftime("%b %d, %Y")

Event Name nsetns
Event Date (YYYY-MM-DD) 2022-10-20


In [ ]:
def input_event_type():
    event_type = input("""
What kind of event is this?

Enter 'w' for Workshop
or    'c' for Competition
""")
    
    event_type = event_type.lower()

    if (event_type not in ['w', 'c']):
        print("Select 'c' or 'c'")
        event_type = input_event_type()
    return event_type
    
event_type = input_event_type()


What kind of event is this?

Enter 'w' for Workshop
or    'c' for Competition
w


# Generate Certificates

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder = "/content/drive/MyDrive/GDSC/Admin/GDSC_Certificates"

fonts_folder = f"{folder}/fonts"

import os
output_folder = f'{folder}/output/{event_name}'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
from PIL import Image, ImageDraw, ImageFont

img = Image.open(f'{folder}/template.png')
W, H = img.size

In [ ]:
def draw_event_details(img, event_type, event_name, event_date):
    im = img.copy()

    
    title = "CERTIFICATE OF "
    text = "has "
    if event_type == 'w':
        # Workshop
        title += "COMPLETION"
        text += "successfully completed the workshop -"
    elif event_type == 'c':
        # Competition
        title += "PARTICIPATION"
        text += "participated in the"
    
    text += f" {event_name}\nheld on {event_date}"
    
    # title
    title_font = ImageFont.truetype(f"{fonts_folder}/Product Sans Bold.ttf", W//30)
    draw = ImageDraw.Draw(im)
    w, h = draw.textsize(title, title_font)
    
    draw.text(
        (
            (W - w)//2,
            (H - h*0.8)/3.5
        ),
        title,
        font = title_font,
        align = "center",
        fill = (235, 66, 52)
    )
    
    # text
    text_font = ImageFont.truetype(f"{fonts_folder}/Product Sans Regular.ttf", W//45)
    draw = ImageDraw.Draw(im)
    w, h = draw.textsize(text, text_font)
    
    draw.text(
        (
            (W - w)/2,
            (H - h*0.8)/1.5
        ),
        text,
        font = text_font,
        align = "center",
        fill = (90, 90, 90)
    )

    if im.mode == 'RGBA':
        im = im.convert('RGB')
    return im

img = draw_event_details(img, event_type, event_name, event_date)
# img.show()

In [ ]:
def draw_name(img, name, y_offset=0):
    im = img.copy()
    font = ImageFont.truetype(f"{fonts_folder}/Product Sans Bold.ttf", W//18)
    
    draw = ImageDraw.Draw(im)
    w, h = draw.textsize(name, font)   
    
    draw.text(
        (
            (W - w)//2,
            (H - h*0.8)//2 + y_offset
        ),
        name,
        font=font,
        align="center",
        fill=(50, 50, 50)
    )

    #im.show()

    if im.mode == 'RGBA':
      im = im.convert('RGB')

    im.save(
        os.path.join(output_folder, f"{name}.pdf")
    )

for name in names:
    draw_name(img, name)

# Send Mails

In [ ]:
from email.message import EmailMessage
import imghdr

In [ ]:
def gen_message(name, receiver_mail, sender_mail):
    msg = EmailMessage()
    msg['From'] = sender_mail
    msg['To'] = receiver_mail
    
    first_name = name.split(" ")[0]
    msg['Subject'] = f"{first_name} - GDSC {event_name} Certificate"

    body = f"""Hi there {first_name}!
Here is your certificate for attending GDSC {event_name} event by Google Developer Student Clubs held on {event_date}.
We hope you enjoyed it, and can't wait to see you at future events as well.

Regards
GDSC Team"""
    
    msg.set_content(body)
    
    file_name = os.path.join(output_folder, f"{name}.pdf")
    with open(file_name, 'rb') as content_file:
        content = content_file.read()
        msg.add_attachment(
            content,
            maintype='application/pdf',
            subtype='pdf',
            filename = name + f" GDSC {event_name} Certificate.pdf"
        )
    
    return msg

In [ ]:
msgs = []
for name, email in zip(names, emails):
    msgs.append(gen_message(name, email, sender_mail))

In [ ]:
msg = msgs[0]
print(msg.get_body())
print("<PDF will be attached below>")

from getpass import getpass
import smtplib

def get_password(message):

  sender_password = getpass(message)

  try:
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_mail, sender_password)
  except:
    get_password("Incorrect password! Type again: ")

  return sender_password

sender_password = get_password("""
---

Do you confirm to send an email like this?
If yes, type password: """)

Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: quoted-printable

Hi there Ahmed!
Here is your certificate for attending GDSC nsetns event by Google Developer =
Student Clubs held on Oct 20, 2022.
We hope you enjoyed it, and can't wait to see you at future events as well.

Regards
GDSC Team

<PDF will be attached below>

---

Do you confirm to send an email like this?
If yes, type password
··········


KeyboardInterrupt: ignored

In [ ]:
import time

with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(sender_mail, sender_password)

    sent_count = 0
    for msg in msgs:
        server.send_message(msg)
        sent_count += 1

        if sent_count % 50 == 0:
          time.sleep(100) # 100 sec delay
        else:
          time.sleep(1) # 1 sec delay

    print("Done :)")